# Deep Learning with Python

## 2.4 The engine of neural networks: gradient-based optimization

> 神经网络的“引擎”: 基于梯度的优化

再看一次我们的第一个神经网络例子中(MNIST)，每一层对输入数据做的运算：

```
output = relu(dot(W, input) + b)
```

这个式子里：W 和 b 是层的属性（权重，或着说可训练参数）。
具体来说，

- `W` 是 kernel 属性；
- `b` 是 bias 属性。

这些「权重」就是神经网络从数据中学习到的东西。

一开始，这些权重被随机初始化成一些较小的值。然后从这次随机的输出开始，反馈调节，逐步改善。

这个改善的过程是在「训练循环」中完成的，只有必要，这个循环可以一直进行下去：

1. 抽取一批训练数据 x 以及对应的 y
2. 向前传播，得到 x 经过网络算出来的预测 y_pred
3. 通过 y_pred 与 y，计算出损失
4. 通过某种方式调整参数，减小损失

前三步都比较简单，第4步更新参数比较复杂，一种比较有效、可行的办法就是利用可微性，通过计算梯度，向梯度的反方向移动参数。

### 导数(derivative)

这一节解释了导数的定义。

(直接去看书吧。)

知道了导数，那要更新 x 来最小化一个函数 `f(x)`，其实只需将 x 向导数的反方向移动。

### 梯度(gradient)

「梯度」是张量运算的导数。或者说「梯度」是「导数」在多元函数上的推广。
某点的梯度代表的是该点的曲率。

考虑:

```
y_pred = dot(W, x)
loss_value = loss(y_pred, y)
```

若固定 x 和 y，则 loss_value 将是一个 W 的函数：

```
loss_value = f(W)
```

设当前点为 `W0`，
则 f 在 W0 的导数(梯度)记为 `gradient(f)(W0)`，
这个梯度值与 W 同型。
其中每个元素 `gradient(f) (W0)[i, j]` 代表改变 `W0[i, j]` 时，f 的变化方向及大小。

所以，要改变 W 的值来实现 `min f`，就可以向梯度的反方向（即**梯度下降**的方向）移动：

```
W1 = W0 - step * gradient(f)(W0)
```


### 随机梯度下降(Stochastic gradient descent)

理论上，给定一个可微函数，其最小值一定在导数为0的点中取到。所以我们只有求出所有导数为0的点，比较其函数值，就可以得到最小值。

这个方法放到神经网络中，就需要解一个关于 `W` 的方程 `gradient(f)(W) = 0`，这是个 N 元方程（N=神经网络中参数个数），而实际上N一般不会少于1k，这使得解这个方程变得几乎不可行。


所以面对这个问题，我们利用上面介绍的4步法，其中第四步使用梯度下降，逐步往梯度的反方向更新参数，一小步一小步地朝减小损失的方向前进：

1. 抽取一批训练数据 x 以及对应的 y
2. 向前传播，得到 x 经过网络算出来的预测 y_pred
3. 通过 y_pred 与 y，计算出损失
4. 通过某种方式调整参数，减小损失
    1. 向后传播，计算损失函数关于网络参数的梯度
    2. 朝梯度的反方向稍微移动参数即可减小损失（W -= step * gradient）

这个方法叫做「小批量随机梯度下降」（mini-batch stochastic gradient descent，mini-batch SGD）。
随机一词是指我们在第1步抽取数据是随机抽取的。

有一些变种的 SGD 不只看当前梯度就更新值了，它们还要看上一次的权重更新。这些变体被称作「优化方法(optimization method)」或者「优化器(optimizer)」。在很多这些变体中，都会使用一个叫「动量(momentum)」的概念。

「动量」主要处理 SGD 中的两个问题：收敛速度和局部极小点。
用动量可以避免在 learning rate 比较小时收敛到局部最优解，而不是向全局最优解继续前进。

这里的动量就是来自物理的那个动量概念。我们可以想象，一个小球在损失曲面上往下(梯度下降的方向)滚，如果有足够的动量，它就可以“冲过”局部最小值，不被困在那里。
在这个例子中，小球的运动不但被当前位置的坡度（当前的加速度）决定，还受当前的速度（这取决于之前的加速度）的影响。

这个思想放到神经网络中，也就是，一次权重值的更新，不但看当前的梯度，还要看上一次权重更新：

```python
# naive implementation of Optimization with momentum
past_velocity = 0.
momentum = 0.1    # Constant momentum factor
while loss > 0.01:    # Optimization loop
    w, loss, gradient = get_current_parameters()
    velocity = past_velocity * momentum + learning_rate * gradient
    w = w + momentum * velocity - learning_rate * gradient
    past_velocity = velocity
    update_parameter(w)
```

### 反向传播算法：链式求导

神经网络是一大堆张量操作链式和在一起的，比如：

```
f(W1, W2, W3) = a(W1, b(W2, c(W3)))    # 其中 W1, W2, W3 是权重
```

微积分里有个「链式法则(chain rule)」可以给这种复合函数求导：`f(g(x)) = f'(g(x)) * g'(x)`

把这个链式法则用到神经网络就搞出了一个叫「反向传播(Backpropagation)」的算法，
这个算法也叫「反式微分(reverse-mode differentiation)」。

反向传播从最终算出的损失出发，从神经网络的最顶层反向作用至最底层，用这个链式法则算出每层里每个参数对于损失的贡献大小。

现在的 TensorFlow 之类的框架，都有种叫「符号微分(symbolic differentiation)」的能力。
这使得这些框架可以自动求出给定神经网络里操作的梯度函数，然后我们就不用手动实现反向传播了（虽然有意思，但写起来真的烦），直接从梯度函数取值就行了。
